# Module 09: Feature Importance and Interpretability

**Difficulty**: ⭐⭐ Intermediate  
**Estimated Time**: 60 minutes  
**Prerequisites**: Module 08 (Feature Selection Methods)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Calculate and interpret permutation importance for any model
2. Extract feature importance from tree-based models
3. Understand SHAP values and their role in model interpretability
4. Compare different feature importance methods
5. Visualize feature importance rankings effectively
6. Use feature importance to improve model understanding and debugging

## 1. Why Feature Importance Matters

**Understanding which features drive predictions is critical for**:
- **Model debugging**: Identify if model learned correct patterns
- **Trust**: Stakeholders need to understand model decisions
- **Feature engineering**: Focus efforts on impactful features
- **Compliance**: Some industries require explainable models
- **Domain insights**: Learn about underlying patterns in data

**Different methods reveal different insights**:
1. **Permutation Importance**: How much does performance drop if we shuffle a feature?
2. **Tree-based Importance**: How often and how much does a feature split data?
3. **Coefficient Magnitude**: How strongly does a feature influence linear models?
4. **SHAP Values**: Individual feature contributions to predictions

In this module, we'll explore these methods and understand when to use each.

## 2. Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes, load_breast_cancer

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor

# Feature importance
from sklearn.inspection import permutation_importance

# Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("✓ Setup complete!")

## 3. Load Dataset

We'll use the diabetes dataset - predicting disease progression from medical measurements.

In [ ]:
# Load diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

print(f"Dataset shape: {X.shape}")
print(f"  - {X.shape[0]} patients")
print(f"  - {X.shape[1]} features")
print(f"\nFeatures: {list(X.columns)}")
print(f"\nTarget: Disease progression (quantitative measure)")
print(f"Target range: {y.min():.1f} to {y.max():.1f}")
print(f"\nFirst few rows:")
X.head()

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
train_score = r2_score(y_train, rf_model.predict(X_train))
test_score = r2_score(y_test, rf_model.predict(X_test))

print(f"Random Forest Performance:")
print(f"  Train R²: {train_score:.3f}")
print(f"  Test R²: {test_score:.3f}")

## 4. Method 1: Tree-Based Feature Importance

**How it works**:
- Trees split on features that reduce impurity most
- Feature importance = total impurity reduction from that feature
- Averaged across all trees in forest

**Advantages**:
- ✅ Fast (calculated during training)
- ✅ Built into tree models
- ✅ Captures non-linear relationships

**Limitations**:
- ❌ Biased toward high-cardinality features
- ❌ Can be unreliable with correlated features
- ❌ Only available for tree-based models

In [ ]:
# Get feature importances from Random Forest
importances_rf = pd.Series(rf_model.feature_importances_, index=X.columns)
importances_rf_sorted = importances_rf.sort_values(ascending=False)

print("Random Forest Feature Importance:")
print(importances_rf_sorted)
print(f"\nSum of importances: {importances_rf_sorted.sum():.3f} (should be 1.0)")

In [ ]:
# Visualize tree-based importance
plt.figure(figsize=(10, 6))
importances_rf_sorted.plot(kind='barh', color='forestgreen', edgecolor='black')
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance\n(Based on Gini Impurity Reduction)', 
         fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"\nTop 3 most important features:")
for i, (feature, importance) in enumerate(importances_rf_sorted.head(3).items(), 1):
    print(f"{i}. {feature}: {importance:.3f}")

## 5. Method 2: Permutation Importance

**How it works**:
1. Train model on original data
2. For each feature:
   - Randomly shuffle that feature's values
   - Measure how much performance drops
3. Larger drop = more important feature

**Advantages**:
- ✅ Works with any model (model-agnostic)
- ✅ Intuitive interpretation
- ✅ Reliable with correlated features
- ✅ Based on actual performance metric

**Limitations**:
- ❌ Slower (requires re-prediction)
- ❌ Can be unstable with small datasets

In [ ]:
# Calculate permutation importance
perm_importance = permutation_importance(
    rf_model, 
    X_test, 
    y_test, 
    n_repeats=10,  # Repeat shuffling 10 times for stability
    random_state=42,
    scoring='r2'
)

# Create dataframe with results
perm_imp_df = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

print("Permutation Importance (on test set):")
print(perm_imp_df)

In [ ]:
# Visualize permutation importance with error bars
fig, ax = plt.subplots(figsize=(10, 6))

y_pos = np.arange(len(perm_imp_df))
ax.barh(y_pos, perm_imp_df['importance_mean'], 
       xerr=perm_imp_df['importance_std'],
       color='steelblue', edgecolor='black', capsize=5)
ax.set_yticks(y_pos)
ax.set_yticklabels(perm_imp_df['feature'])
ax.set_xlabel('Importance (Drop in R² when shuffled)')
ax.set_title('Permutation Importance with Standard Deviation', 
            fontsize=12, fontweight='bold')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("Interpretation: If shuffling 'bmi' drops R² by 0.3, then 'bmi' explains ~30% of model performance")

## 6. Method 3: Linear Model Coefficients

**For linear models**: Feature importance = absolute coefficient value

**Important**: Must standardize features first for fair comparison!

In [ ]:
# Standardize features (important for coefficient comparison!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train linear regression
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

# Get coefficients
coefficients = pd.Series(linear_model.coef_, index=X.columns)
coefficients_abs = coefficients.abs().sort_values(ascending=False)

print("Linear Regression Coefficients (on standardized features):")
print(coefficients.sort_values(key=abs, ascending=False))

# Model performance
linear_score = r2_score(y_test, linear_model.predict(X_test_scaled))
print(f"\nLinear Model Test R²: {linear_score:.3f}")

In [ ]:
# Visualize coefficients
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Raw coefficients (with sign)
coefficients_sorted = coefficients.sort_values()
colors = ['red' if x < 0 else 'green' for x in coefficients_sorted.values]
axes[0].barh(range(len(coefficients_sorted)), coefficients_sorted.values, 
            color=colors, edgecolor='black')
axes[0].set_yticks(range(len(coefficients_sorted)))
axes[0].set_yticklabels(coefficients_sorted.index)
axes[0].set_xlabel('Coefficient Value')
axes[0].set_title('Linear Model Coefficients\n(Red=Negative, Green=Positive)', 
                  fontweight='bold')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
axes[0].grid(True, alpha=0.3, axis='x')

# Absolute values (importance)
axes[1].barh(range(len(coefficients_abs)), coefficients_abs.values, 
            color='coral', edgecolor='black')
axes[1].set_yticks(range(len(coefficients_abs)))
axes[1].set_yticklabels(coefficients_abs.index)
axes[1].set_xlabel('Absolute Coefficient Value')
axes[1].set_title('Feature Importance\n(Absolute Coefficients)', fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("Positive coefficient = feature increases target")
print("Negative coefficient = feature decreases target")

## 7. Method 4: Correlation with Target

**Simple but useful**: How strongly does each feature correlate with the target?

**Limitations**: Only captures linear relationships!

In [ ]:
# Calculate correlation with target
correlations = X_train.corrwith(pd.Series(y_train, index=X_train.index))
correlations_abs = correlations.abs().sort_values(ascending=False)

print("Correlation with Target:")
print(correlations.sort_values(key=abs, ascending=False))

# Visualize
plt.figure(figsize=(10, 6))
correlations_sorted = correlations.sort_values()
colors = ['red' if x < 0 else 'green' for x in correlations_sorted.values]
plt.barh(range(len(correlations_sorted)), correlations_sorted.values, 
        color=colors, edgecolor='black')
plt.yticks(range(len(correlations_sorted)), correlations_sorted.index)
plt.xlabel('Correlation with Target')
plt.title('Feature Correlation with Disease Progression', fontsize=12, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 8. Compare All Methods

Different methods can give different rankings! Let's compare them side-by-side.

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Feature': X.columns,
    'RF Importance': importances_rf,
    'Permutation Imp': perm_importance.importances_mean,
    'Linear Coef (abs)': coefficients.abs(),
    'Correlation (abs)': correlations.abs()
})

# Rank each method (1 = most important)
for col in ['RF Importance', 'Permutation Imp', 'Linear Coef (abs)', 'Correlation (abs)']:
    comparison_df[f'{col} Rank'] = comparison_df[col].rank(ascending=False)

print("Feature Importance Comparison:")
print("="*80)
print(comparison_df.sort_values('RF Importance', ascending=False))

In [ ]:
# Visualize ranking comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

methods = [
    ('RF Importance', 'forestgreen', 'Random Forest Importance'),
    ('Permutation Imp', 'steelblue', 'Permutation Importance'),
    ('Linear Coef (abs)', 'coral', 'Linear Coefficients (abs)'),
    ('Correlation (abs)', 'purple', 'Correlation with Target (abs)')
]

for idx, (col, color, title) in enumerate(methods):
    row = idx // 2
    col_idx = idx % 2
    
    data = comparison_df.sort_values(col, ascending=False)
    axes[row, col_idx].barh(range(len(data)), data[col].values, 
                            color=color, edgecolor='black')
    axes[row, col_idx].set_yticks(range(len(data)))
    axes[row, col_idx].set_yticklabels(data['Feature'])
    axes[row, col_idx].set_xlabel('Importance Score')
    axes[row, col_idx].set_title(title, fontweight='bold')
    axes[row, col_idx].invert_yaxis()
    axes[row, col_idx].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nNotice: Different methods prioritize different features!")
print("- Tree methods capture non-linear patterns")
print("- Linear methods capture linear relationships")
print("- Permutation shows actual impact on predictions")

In [ ]:
# Calculate rank correlation between methods
rank_cols = ['RF Importance Rank', 'Permutation Imp Rank', 
             'Linear Coef (abs) Rank', 'Correlation (abs) Rank']
rank_corr = comparison_df[rank_cols].corr()

print("Rank Correlation Between Methods:")
print("(1.0 = perfect agreement, 0.0 = no agreement)")
print(rank_corr)

# Visualize correlation
plt.figure(figsize=(10, 8))
sns.heatmap(rank_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0,
           square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Agreement Between Feature Importance Methods\n(Rank Correlation)', 
         fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nHigh correlation = methods agree on important features")
print("Low correlation = methods disagree (different perspectives!)")

## 9. Introduction to SHAP Values

**SHAP (SHapley Additive exPlanations)** provides individual feature contributions:
- Not just global importance, but per-prediction explanations
- Based on game theory (Shapley values)
- Shows both magnitude and direction of impact

**Note**: SHAP requires external library (`shap`). Here we demonstrate the concept.

In [ ]:
# Conceptual SHAP demonstration (simplified)
print("SHAP Values - Conceptual Overview\n")
print("While other methods give global feature importance,")
print("SHAP values explain individual predictions.\n")

print("Example prediction breakdown:")
print("="*60)
print("Patient ID: 42")
print("Predicted disease progression: 185")
print("\nFeature contributions (SHAP values):")
print("  Base value (average): 150")
print("  + bmi contribution: +30")
print("  + bp contribution: +15")
print("  + s5 contribution: -10")
print("  + other features: 0")
print("  = Final prediction: 185")
print("="*60)

print("\nSHAP advantages:")
print("✅ Individual prediction explanations")
print("✅ Shows positive/negative contributions")
print("✅ Theoretically sound (game theory)")
print("✅ Works with any model")

print("\nTo use SHAP in practice:")
print("  pip install shap")
print("  import shap")
print("  explainer = shap.TreeExplainer(model)")
print("  shap_values = explainer.shap_values(X)")

## 10. Exercise Section

### Exercise 1: Feature Importance for Classification

Apply feature importance methods to the breast cancer classification dataset.

In [ ]:
# Exercise 1: Classification feature importance

# Load breast cancer dataset
cancer = load_breast_cancer()
X_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y_cancer = cancer.target

print(f"Breast Cancer Dataset: {X_cancer.shape}")
print(f"Features: {list(X_cancer.columns[:5])} ...")

# TODO:
# 1. Split the data
# 2. Train a Random Forest Classifier
# 3. Extract feature importance
# 4. Calculate permutation importance
# 5. Compare the top 5 features from each method

# Your code here:


In [ ]:
# Solution to Exercise 1

# 1. Split data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cancer, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
)

# 2. Train Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_c, y_train_c)
print(f"Test Accuracy: {rf_clf.score(X_test_c, y_test_c):.3f}\n")

# 3. Feature importance
rf_imp = pd.Series(rf_clf.feature_importances_, index=X_cancer.columns).sort_values(ascending=False)

# 4. Permutation importance
perm_imp = permutation_importance(rf_clf, X_test_c, y_test_c, 
                                  n_repeats=10, random_state=42)
perm_imp_series = pd.Series(perm_imp.importances_mean, 
                           index=X_cancer.columns).sort_values(ascending=False)

# 5. Compare top 5
print("Top 5 by Random Forest Importance:")
print(rf_imp.head(5))
print("\nTop 5 by Permutation Importance:")
print(perm_imp_series.head(5))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

rf_imp.head(10).plot(kind='barh', ax=axes[0], color='forestgreen', edgecolor='black')
axes[0].set_title('Top 10 by RF Importance', fontweight='bold')
axes[0].invert_yaxis()

perm_imp_series.head(10).plot(kind='barh', ax=axes[1], color='steelblue', edgecolor='black')
axes[1].set_title('Top 10 by Permutation Importance', fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

### Exercise 2: Verify Feature Importance

Manually verify that removing top features hurts performance more than removing bottom features.

In [ ]:
# Exercise 2: Verify importance by removal

# Using the diabetes dataset
# TODO:
# 1. Train model with ALL features
# 2. Train model WITHOUT top 3 most important features
# 3. Train model WITHOUT bottom 3 least important features
# 4. Compare R² scores
#
# Expected: Removing top features hurts more!

# Your code here:


In [ ]:
# Solution to Exercise 2

# Get importance ranking
importance_ranking = importances_rf_sorted
top_3 = importance_ranking.head(3).index.tolist()
bottom_3 = importance_ranking.tail(3).index.tolist()

print(f"Top 3 features: {top_3}")
print(f"Bottom 3 features: {bottom_3}\n")

# 1. All features
model_all = RandomForestRegressor(n_estimators=100, random_state=42)
model_all.fit(X_train, y_train)
r2_all = r2_score(y_test, model_all.predict(X_test))

# 2. Without top 3
features_no_top = [f for f in X.columns if f not in top_3]
model_no_top = RandomForestRegressor(n_estimators=100, random_state=42)
model_no_top.fit(X_train[features_no_top], y_train)
r2_no_top = r2_score(y_test, model_no_top.predict(X_test[features_no_top]))

# 3. Without bottom 3
features_no_bottom = [f for f in X.columns if f not in bottom_3]
model_no_bottom = RandomForestRegressor(n_estimators=100, random_state=42)
model_no_bottom.fit(X_train[features_no_bottom], y_train)
r2_no_bottom = r2_score(y_test, model_no_bottom.predict(X_test[features_no_bottom]))

# 4. Compare
print("Model Performance Comparison:")
print("="*50)
print(f"All features:              R² = {r2_all:.3f}")
print(f"Without TOP 3 features:    R² = {r2_no_top:.3f} (drop: {r2_all - r2_no_top:.3f})")
print(f"Without BOTTOM 3 features: R² = {r2_no_bottom:.3f} (drop: {r2_all - r2_no_bottom:.3f})")
print("="*50)
print(f"\nRemoving top features hurts {(r2_all - r2_no_top) / (r2_all - r2_no_bottom):.1f}x more!")
print("This confirms our importance rankings are meaningful.")

### Exercise 3: Interpret Model Decisions

Use feature importance to understand what drives predictions for specific samples.

In [ ]:
# Exercise 3: Interpret specific predictions

# Pick 3 test samples
sample_indices = [0, 10, 20]

# TODO:
# 1. Show actual features values for these samples
# 2. Show predicted vs actual target
# 3. Identify which features are "unusual" (far from mean)
# 4. Relate feature values to importance rankings

# Your code here:


In [ ]:
# Solution to Exercise 3

sample_indices = [0, 10, 20]

for idx in sample_indices:
    sample = X_test.iloc[idx]
    actual = y_test.iloc[idx]
    predicted = rf_model.predict(sample.values.reshape(1, -1))[0]
    
    print(f"\n{'='*60}")
    print(f"Sample {idx}")
    print(f"{'='*60}")
    print(f"Actual disease progression: {actual:.1f}")
    print(f"Predicted: {predicted:.1f}")
    print(f"Error: {abs(actual - predicted):.1f}\n")
    
    # Show feature values compared to mean
    print("Feature values (deviation from training mean):")
    train_mean = X_train.mean()
    deviations = (sample - train_mean).abs().sort_values(ascending=False)
    
    print("\nTop 5 most unusual features (far from average):")
    for feature in deviations.head(5).index:
        value = sample[feature]
        mean_val = train_mean[feature]
        importance = importances_rf[feature]
        print(f"  {feature}: {value:.3f} (mean: {mean_val:.3f}, importance: {importance:.3f})")

print("\n" + "="*60)
print("Interpretation:")
print("- High values in important features → higher predictions")
print("- Look for unusual values in top features to explain predictions")
print("="*60)

## 11. Summary

### Key Takeaways

1. **Feature importance helps us understand models**
   - Debug: Are we learning the right patterns?
   - Trust: Explain predictions to stakeholders
   - Improve: Focus engineering on important features

2. **Four main methods for feature importance**:
   - **Tree-based**: Built-in, fast, tree-specific
   - **Permutation**: Model-agnostic, intuitive, slower
   - **Coefficients**: Linear models only, interpretable
   - **SHAP**: Individual predictions, theoretically sound

3. **Different methods reveal different insights**:
   - May disagree on rankings
   - Linear vs non-linear patterns
   - Global vs local importance

4. **Best practices**:
   - Use multiple methods to get full picture
   - Standardize features for linear models
   - Verify importance by removal experiments
   - Consider domain knowledge

### When to Use Each Method

**Tree-based Importance**:
- ✅ Using tree models (RF, XGBoost)
- ✅ Need fast results
- ✅ Global feature ranking
- ⚠️  Can be biased with correlated features

**Permutation Importance**:
- ✅ Any model type
- ✅ Want model-agnostic method
- ✅ Correlated features present
- ❌ Slow for large datasets

**Linear Coefficients**:
- ✅ Linear/logistic regression
- ✅ Need interpretable weights
- ✅ Understand direction of effect
- ❌ Only linear relationships

**SHAP Values**:
- ✅ Need individual prediction explanations
- ✅ Compliance/regulatory requirements
- ✅ Want theoretically sound method
- ❌ Requires additional library
- ❌ Computationally expensive

### Practical Workflow

1. **Start with built-in importance** (tree-based or coefficients)
2. **Validate with permutation importance**
3. **Deep dive with SHAP** for critical features
4. **Verify with domain experts**
5. **Test by removal** to confirm impact

### What's Next?

**Module 10**: Automated Feature Engineering - Learn to use tools like featuretools for automatic feature creation

### Additional Resources

- [SHAP Documentation](https://shap.readthedocs.io/)
- [Permutation Importance](https://scikit-learn.org/stable/modules/permutation_importance.html)
- [Interpretable ML Book](https://christophm.github.io/interpretable-ml-book/)

---

**Congratulations!** You've completed Module 09. You now understand:
- How to calculate feature importance with multiple methods
- When to use each importance method
- How different methods can give different insights
- How to interpret and visualize feature importance
- How to verify importance rankings experimentally

Ready to explore automated feature engineering? Let's move to **Module 10: Automated Feature Engineering**!